This notebook compares the overfitting of Fairlearn Vs OxonFair using random forests and decision trees on the adult dataset.

We use sex as the protected attribute.

Even on this low-dimensional data, the default parameters of scikit-learn cause both decision trees and random forests to overfit. 

The models obtain 0 error on the training set. As a consequence of this, defintions such as equal opportunity are trivially satisfied, and fairness methods such as fairlearn which enforce fairness on the training set do not work.

This overfitting, and the consequential failure of fairness methods to work can be avoided by specifying a low maximimal tree depth. The examples in Fairlearn documentation typically use a tree depth of 4 on adult. 

Oxonfair allows for the enforcing of fairness on validation data, and this means that it can enforce fairness even when the training error is zero. 

In [1]:
from oxonfair import FairPredictor, performance, dataset_loader
from oxonfair import group_metrics as gm
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train,val,test = dataset_loader.adult()
basetree = DecisionTreeClassifier().fit(X=train['data'], y=train['target'])
baseforest = RandomForestClassifier().fit(X=train['data'], y=train['target'])

We now specify a fair predictors over the validation set.

In [3]:
# The outputs of a decision tree are all 0 or 1, so we add Gaussian noise to allow thresholding to work
ftree=FairPredictor(basetree,val,add_noise=0.001)
fforest=FairPredictor(baseforest,val)

We call fit to enforce equal opportunity.

In [4]:
ftree.fit(gm.accuracy,gm.equal_opportunity,0.02)
fforest.fit(gm.accuracy,gm.equal_opportunity,0.02)

We now focus on trees only.
And evaluate fairness on validation data.

In [5]:
ftree.evaluate_fairness()

,original,updated
Statistical Parity,0.197600,0.154953
Predictive Parity,0.085417,0.090281
Equal Opportunity,0.093654,0.007088
Average Group Difference in False Negative Rate,0.093654,0.007088
Equalized Odds,0.101198,0.046011
Conditional Use Accuracy,0.095122,0.110521
Average Group Difference in Accuracy,0.126237,0.135298
Treatment Equality,0.034893,0.294955


And on the test set.

In [6]:
ftree.evaluate_fairness(test)

,original,updated
Statistical Parity,0.199639,0.155288
Predictive Parity,0.082702,0.086645
Equal Opportunity,0.091534,0.003228
Average Group Difference in False Negative Rate,0.091534,0.003228
Equalized Odds,0.101541,0.044881
Conditional Use Accuracy,0.094067,0.109457
Average Group Difference in Accuracy,0.128169,0.137476
Treatment Equality,0.039490,0.296451


We now check validation performance.

In [7]:
ftree.evaluate()

,original,updated
Accuracy,0.814824,0.807699
Balanced Accuracy,0.746927,0.717614
F1 score,0.614493,0.575560
MCC,0.492646,0.452963
Precision,0.612300,0.609962
Recall,0.616701,0.544832
ROC AUC,0.747207,0.708037


And on the test set.

In [8]:
ftree.evaluate(test)

,original,updated
Accuracy,0.813529,0.806814
Balanced Accuracy,0.746659,0.719374
F1 score,0.613478,0.577467
MCC,0.490640,0.453435
Precision,0.608622,0.605787
Recall,0.618412,0.551677
ROC AUC,0.746638,0.703743


We now run fairlearn on the same data.

In [9]:
from fairlearn.reductions import TruePositiveRateParity, ExponentiatedGradient
mitagator = ExponentiatedGradient(DecisionTreeClassifier(),TruePositiveRateParity())
mitagator.fit(X=train['data'],y=train['target'],sensitive_features=train['data']['sex'])

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:214: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.pos_basis[i]["+", e, g] = 1
/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fai

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.TruePositiveRateParity object at 0x302069120>,
                      estimator=DecisionTreeClassifier(),
                      nu=2.0474182056426843e-05)

To evaluate fairlearn, we write a helper function to evaluate performance and fairness on train or test, and concat the outputs together.  

In [10]:
def eval(train, classifier=mitagator):
    return pd.concat((performance.evaluate(train['target'], classifier.predict(train['data'])),
                      performance.evaluate_fairness(train['target'], classifier.predict(train['data']), train['groups'])),axis=0)

out = pd.concat((eval(train), eval(test)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,0.999959,0.812300
Balanced Accuracy,0.999973,0.744444
F1 score,0.999914,0.610337
MCC,0.999888,0.486736
Precision,0.999829,0.606419
Recall,1.000000,0.614305
ROC AUC,0.999973,0.744444
Statistical Parity,0.194454,0.197058
Predictive Parity,0.001130,0.083314
Equal Opportunity,0.000000,0.084030


Evaluating the initially trained baseline classifier we find that, as expected, fairlearn did not substantially alter the performance or unfairness of the classifier (beyond altering the random seed of the tree).

In [11]:
out = pd.concat((eval(train, basetree), eval(test, basetree)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,0.999959,0.813529
Balanced Accuracy,0.999914,0.746659
F1 score,0.999914,0.613478
MCC,0.999888,0.490640
Precision,1.000000,0.608622
Recall,0.999829,0.618412
ROC AUC,0.999914,0.746659
Statistical Parity,0.194701,0.199639
Predictive Parity,0.000000,0.082702
Equal Opportunity,0.001131,0.091534


We now do the same with the random forest classifier.

In [12]:
fforest.evaluate_fairness()

,original,updated
Statistical Parity,0.175160,0.142618
Predictive Parity,0.016260,0.069048
Equal Opportunity,0.089139,0.012442
Average Group Difference in False Negative Rate,0.089139,0.012442
Equalized Odds,0.078731,0.029162
Conditional Use Accuracy,0.056203,0.089918
Average Group Difference in Accuracy,0.104785,0.103921
Treatment Equality,0.142459,0.153912


In [13]:
fforest.evaluate_fairness(test)

,original,updated
Statistical Parity,0.172043,0.143441
Predictive Parity,0.002382,0.033366
Equal Opportunity,0.056185,0.018006
Average Group Difference in False Negative Rate,0.056185,0.018006
Equalized Odds,0.065560,0.036851
Conditional Use Accuracy,0.054140,0.076282
Average Group Difference in Accuracy,0.116769,0.117395
Treatment Equality,0.117046,0.127107


In [14]:
fforest.evaluate()

,original,updated
Accuracy,0.858067,0.857903
Balanced Accuracy,0.775585,0.766330
F1 score,0.675529,0.665510
MCC,0.590005,0.584675
Precision,0.745763,0.762031
Recall,0.617385,0.590691
ROC AUC,0.908913,0.897145


In [15]:
fforest.evaluate(test)

,original,updated
Accuracy,0.850053,0.851364
Balanced Accuracy,0.765036,0.758273
F1 score,0.657693,0.651163
MCC,0.566717,0.565454
Precision,0.724763,0.742657
Recall,0.601985,0.579740
ROC AUC,0.899995,0.885634


In [16]:
mitagator = ExponentiatedGradient(RandomForestClassifier(),TruePositiveRateParity())
mitagator.fit(X=train['data'],y=train['target'],sensitive_features=train['data']['sex'])

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:214: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.pos_basis[i]["+", e, g] = 1
/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fai

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.TruePositiveRateParity object at 0x303c3cbe0>,
                      estimator=RandomForestClassifier(),
                      nu=3.5460871351809455e-05)

In [17]:
out = pd.concat((eval(train,mitagator), eval(test,mitagator)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,0.999959,0.849316
Balanced Accuracy,0.999973,0.766428
F1 score,0.999914,0.658627
MCC,0.999888,0.566129
Precision,0.999829,0.719206
Recall,1.000000,0.607461
ROC AUC,0.999973,0.766428
Statistical Parity,0.194454,0.175957
Predictive Parity,0.001130,0.010290
Equal Opportunity,0.000000,0.051974
